In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 8.1MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
     |████████████████████████████████| 2.9MB 49.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=cef34c6168643fb77b86d4517e7e71a99441a6a9b6d82ec91c4aaed4123561cf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ast
import re

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/ARC/df_easy_train_3.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/ARC/df_easy_dev_3.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ARC/df_easy_test_3.csv')

In [ ]:
train_df.head()

,AnswerKey,question,only_question,quesiton_length,only_answers,context,options_list,option_1_docs,option_2_docs,option_3_docs,option_4_docs,option_5_docs
0,B,Which factor will most likely cause a person t...,Which factor will most likely cause a person t...,66,(A) a leg muscle relaxing after exercise (B) a...,Piercings show that a person is rebelious and ...,"['a leg muscle relaxing after exercise ', 'bac...","{""-' Which joint of the leg con- tains most mu...",{'When the larvae of Trichinella worms get int...,{'Environmental perturbations are most likely ...,{'Carbohydrates will most likely be the larges...,{'Which technique was most likely used to deve...
1,B,Lichens are symbiotic organisms made of green ...,Lichens are symbiotic organisms made of green ...,139,(A) carbon dioxide (B) food (C) protection (D)...,Lichens are comprised of blue-green or green a...,"['carbon dioxide ', 'food ', 'protection ', 'w...","{""The hydras let these algae live inside them ...","{'algae near the surface of ponds, lakes, and ...","{""In this symbiotic relationship, the algae ga...",{'This is a small genus of marine and brackish...,{'Lichens are comprised of blue-green or green...
2,D,When a switch is used in an electrical circuit...,When a switch is used in an electrical circuit...,63,(A) cause the charge to build. (B) increase an...,Rotary switch A ` rotary switch ' is an electr...,"['cause the charge to build. ', 'increase and ...",{'If an ordinary switch is used to control the...,{'And when the electrical switch was turned OF...,{'If an ordinary switch is used to control the...,{'When the circuit switch is open (Figure 2) c...,{'If an ordinary switch is used to control the...
3,A,Which of the following is an example of an ass...,Which of the following is an example of an ass...,61,(A) contact lens (B) motorcycle (C) raincoat (...,Which of the following is an example of an ang...,"['contact lens ', 'motorcycle ', 'raincoat ', ...",{'An instrument for measuring the degree of cu...,{'The characters are typically members of an o...,"{'[DUSSEL hurriedly takes off the raincoat, gi...",{'coffee\tn\ta beverage consisting of an infus...,{'Which of the following is an example of an a...
4,3,"Rocks are classified as igneous, metamorphic, ...","Rocks are classified as igneous, metamorphic, ...",74,(1) their color (2) their shape (3) how they f...,"In one simplistic and linear example, elements...","['their color ', 'shape ', 'how they formed ',...","{'In one simplistic and linear example, elemen...",{'Western carrot cultivars are commonly classi...,"{'- formed by “changing” preexisting igneous, ...",{'This group of minerals is of common occur- r...,"{'The second class of rocks, sedimentary,\xa0 ..."


In [ ]:
len(a.keys())

50

In [ ]:
train_df.shape, dev_df.shape, test_df.shape

((2251, 12), (570, 12), (2376, 12))

In [ ]:
import torch

In [ ]:
from torch.nn import Softmax

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

In [ ]:
device = 'cuda'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
drd_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/DRD_cpu').to(device)
ard_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/ARD_cpu').to(device)

In [ ]:
def scorer(question, option, document):
    '''
    This function will take 3 things: question, option and document.

    Then it will pass question + document into DRD model and get a score.
    It will also pass question + option + document into ARD model and get another score.
    
    This function returns DRD and ARD scores, i.e., logits values for label 1.
    '''
    ard_input_string = f'[CLS] {question} [SEP] {option} [SEP] {document} [SEP]'

    drd_inputs = tokenizer(question, document, truncation='only_second', max_length=512, add_special_tokens=True, padding='max_length', return_tensors='pt')
    drd_inputs = drd_inputs.to(device)

    drd_outputs = drd_model(**drd_inputs)
    drd_softmax = Softmax(dim=1)(drd_outputs.logits)
    drd_score = drd_softmax.detach().cpu().numpy()[0][1]

    ard_input_ids = tokenizer.encode(ard_input_string, add_special_tokens=False, truncation=True, max_length=512, return_tensors='pt')
    ard_attention_mask = torch.ones([1,ard_input_ids.size()[1]])

    ard_input_ids = ard_input_ids.to(device)
    ard_attention_mask = ard_attention_mask.to(device)
    ard_outputs = ard_model(input_ids=ard_input_ids, attention_mask=ard_attention_mask)
    ard_softmax = Softmax(dim=1)(ard_outputs.logits)
    ard_score = ard_softmax.detach().cpu().numpy()[0][1]

    return drd_score, ard_score

In [ ]:
%time drd_score, srd_score = scorer('asds','asda','sdsad')

CPU times: user 50.1 ms, sys: 19.4 ms, total: 69.5 ms
Wall time: 70 ms


In [ ]:
def context_retreiver_2(dataset):
    '''
    This function will return context for every datapoint in the dataset.
    '''
    question = dataset['only_question']
    options_list = ast.literal_eval(dataset['options_list']) 

    scores = dict()
    for j in range(len(options_list)):
        if j == 0:
            current_option = options_list[0]

            option_1_docs = ast.literal_eval(dataset['option_1_docs'])
            option_1_docs_keys = list(option_1_docs.keys())
            option_1_docs_values = list(option_1_docs.values())
            option_1_docs_values = [float(i)+1e-6/(sum(option_1_docs_values)+1e-6) for i in option_1_docs_values]

            for k in range(len(option_1_docs_keys)):
                current_doc = option_1_docs_keys[k]
                current_doc = re.sub('\s+',' ', current_doc)
                drd_score, ard_score = scorer(question, current_option, current_doc)

                tf_idf_score = option_1_docs_values[k]

                total_score = tf_idf_score + drd_score + ard_score

                if current_doc not in scores.keys():
                    scores[current_doc] = total_score
                else:
                    if total_score > scores[current_doc]:
                        scores[current_doc] = total_score
                    else:
                        continue
        elif j == 1:
            current_option = options_list[1]
            option_2_docs = ast.literal_eval(dataset['option_2_docs'])
            option_2_docs_keys = list(option_2_docs.keys())
            option_2_docs_values = list(option_2_docs.values())
            option_2_docs_values = [float(i)+1e-6/(sum(option_2_docs_values)+1e-6) for i in option_2_docs_values]

            for k in range(len(option_2_docs_keys)):
                current_doc = option_2_docs_keys[k]
                current_doc = re.sub('\s+',' ', current_doc)
                drd_score, ard_score = scorer(question, current_option, current_doc)

                tf_idf_score = option_2_docs_values[k]

                total_score = tf_idf_score + drd_score + ard_score

                if current_doc not in scores.keys():
                    scores[current_doc] = total_score
                else:
                    if total_score > scores[current_doc]:
                        scores[current_doc] = total_score
                    else:
                        continue
        elif j == 2:
            current_option = options_list[2]
            option_3_docs = ast.literal_eval(dataset['option_3_docs'])
            option_3_docs_keys = list(option_3_docs.keys())
            option_3_docs_values = list(option_3_docs.values())
            option_3_docs_values = [float(i)+1e-6/(sum(option_3_docs_values)+1e-6) for i in option_3_docs_values]

            for k in range(len(option_3_docs_keys)):
                current_doc = option_3_docs_keys[k]
                current_doc = re.sub('\s+',' ', current_doc)
                drd_score, ard_score = scorer(question, current_option, current_doc)

                tf_idf_score = option_3_docs_values[k]

                total_score = tf_idf_score + drd_score + ard_score

                if current_doc not in scores.keys():
                    scores[current_doc] = total_score
                else:
                    if total_score > scores[current_doc]:
                        scores[current_doc] = total_score
                    else:
                        continue
        elif j == 3:
            current_option = options_list[3]
            option_4_docs = ast.literal_eval(dataset['option_4_docs'])
            option_4_docs_keys = list(option_4_docs.keys())
            option_4_docs_values = list(option_4_docs.values())
            option_4_docs_values = [float(i)+1e-6/(sum(option_4_docs_values)+1e-6) for i in option_4_docs_values]

            for k in range(len(option_4_docs_keys)):
                current_doc = option_4_docs_keys[k]
                current_doc = re.sub('\s+',' ', current_doc)
                drd_score, ard_score = scorer(question, current_option, current_doc)

                tf_idf_score = option_4_docs_values[k]

                total_score = tf_idf_score + drd_score + ard_score

                if current_doc not in scores.keys():
                    scores[current_doc] = total_score
                else:
                    if total_score > scores[current_doc]:
                        scores[current_doc] = total_score
                    else:
                        continue
        elif j == 4:
            current_option = options_list[4]
            option_5_docs = ast.literal_eval(dataset['option_5_docs'])
            option_5_docs_keys = list(option_5_docs.keys())
            option_5_docs_values = list(option_5_docs.values())
            option_5_docs_values = [float(i)+1e-6/(sum(option_5_docs_values)+1e-6) for i in option_5_docs_values]

            for k in range(len(option_5_docs_keys)):
                current_doc = option_5_docs_keys[k]
                current_doc = re.sub('\s+',' ', current_doc)
                drd_score, ard_score = scorer(question, current_option, current_doc)

                tf_idf_score = option_5_docs_values[k]

                total_score = tf_idf_score + drd_score + ard_score

                if current_doc not in scores.keys():
                    scores[current_doc] = total_score
                else:
                    if total_score > scores[current_doc]:
                        scores[current_doc] = total_score
                    else:
                        continue

        context = []
        for l, w in enumerate(sorted(scores, key=scores.get, reverse=True)):
            if l == 20:
                break
            else:
                context.append(w)
    joined_context = ' '.join(i for i in context)
    return joined_context

In [ ]:
list_of_series = []

for i in range(len(train_df)):
    list_of_series.append(train_df.iloc[i])

In [ ]:
contexts = []

for i in range(len(list_of_series)):
    context = context_retreiver_2(list_of_series[i])
    contexts.append(context)
    print(i)

In [ ]:
train_df['context'] = contexts

In [ ]:
train_df.to_csv('/content/drive/MyDrive/arc_train_context', index=False)